In [1]:
%load_ext autoreload

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np

/Users/matsalexander/.pyenv/versions/3.10.13/envs/gluon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%autoreload
import sys
sys.path.append('../')
from pipeline import Pipin
pipin = Pipin()

Current working directory: /Users/matsalexander/Desktop/Forest Gump/mats/gluon


In [21]:
df0 = pipin.get_combined_datasets()

In [35]:
sne = df0["snow_density:kgm3"]
# df0.drop(columns=["snow_density:kgm3"], inplace=True)
# find all nan values in sne
sne_nan_mask = np.isnan(sne)
print("rader med snow_density", len(sne[~sne_nan_mask]))
# sne.head(15)

rader med snow_density 760


In [36]:
snow_columns = df0.columns[df0.columns.str.contains("snow")]
df_snow = df0[snow_columns]
for col in df_snow.columns:
    df_snow[col].fillna(0, inplace=True)
# df_snow.head(15)

df_rows_with_snow_and_sum_greater_than_zero = df_snow[(df_snow.sum(axis=1) > 0).values]

# for nan values set to 0

for col in df_snow.columns:
    print(col, df_snow[col].sum())
for col in df_snow.columns:
    print(col)
df_rows_with_snow_and_sum_greater_than_zero
# df0.drop(columns=["snow_melt_10min:mm"], inplace=True)
# df0.drop(columns=["snow_drift:idx"], inplace=True)
sum_snow_density = df0[df0["snow_density:kgm3"] > 0]["snow_density:kgm3"]
sum_snow_density



snow_melt_10min:mm 7.7375
snow_density:kgm3 190000.0
fresh_snow_6h:cm 727.2
fresh_snow_1h:cm 88.15
snow_water:kgm2 3471.5
fresh_snow_12h:cm 2321.375
fresh_snow_3h:cm 290.175
fresh_snow_24h:cm 5061.1997
snow_depth:cm 2754.225
snow_drift:idx 0.0
snow_melt_10min:mm
snow_density:kgm3
fresh_snow_6h:cm
fresh_snow_1h:cm
snow_water:kgm2
fresh_snow_12h:cm
fresh_snow_3h:cm
fresh_snow_24h:cm
snow_depth:cm
snow_drift:idx


/var/folders/b0/slnc__2x31jczblnsfgfmjw00000gn/T/ipykernel_30830/230111060.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_snow[col].fillna(0, inplace=True)


3098     250.0
3099     250.0
3100     250.0
3101     250.0
3102     250.0
         ...  
47381    250.0
47382    250.0
47383    250.0
47384    250.0
47385    250.0
Name: snow_density:kgm3, Length: 760, dtype: float32

In [5]:
df, df_tune = pipin.split_train_tune(df0)


In [9]:
train, tune = TabularDataset(df), TabularDataset(df_tune)

In [10]:
label="pv_measurement"
eval_metric="mean_absolute_error"

In [11]:
predictor1 = TabularPredictor(label=label, eval_metric=eval_metric).fit(
    train,
    presets='best_quality', 
    tuning_data = tune,
    use_bag_holdout=True,

    )

No path specified. Models will be saved in: "AutogluonModels/ag-20231105_175339/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231105_175339/"
AutoGluon Version:  0.8.2
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 22.5.0: Thu Jun  8 22:22:20 PDT 2023; root:xnu-8796.121.3~7/RELEASE_ARM64_T6000
Disk Space Avail:   741.06 GB / 994.66 GB (74.5%)
Train Data Rows:    45672
Train Data Columns: 49
Tuning Data Rows:    45697
Tuning Data Columns: 49
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (5733.42, 0.0, 593.02762, 1038.32712)
	If 'regression' is not the correct problem_t

In [12]:
test_data = pipin.get_combined_test_data()
test_data = TabularDataset(test_data)


In [13]:

pred = pd.DataFrame(predictor1.predict(test_data))

negatives_pred = pred[pred["pv_measurement"] < 0]
negatives_pred.shape

(265, 1)

In [14]:
from utils import get_unique_filename
final_prediction = pipin.post_processing(pred, prediction_column="pv_measurement")
final_prediction.to_csv(get_unique_filename('gluon_tuned'))

In [15]:
diff = pipin.compare_mae(final_prediction)
diff

94.53856679993704

In [16]:
predictor1.evaluate(df, silent=True)

{'mean_absolute_error': -13.538724429124958,
 'root_mean_squared_error': -33.11081092401292,
 'mean_squared_error': -1096.3258000457333,
 'r2': 0.9989830941481787,
 'pearsonr': 0.9995249843111538,
 'median_absolute_error': -2.5812615966796786}